# Google drive mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/open_directory/test/IMAGE

/content/drive/MyDrive/open_directory/test/IMAGE


In [ ]:
!pwd

/content/drive/MyDrive/open_directory/test/IMAGE


# DATA LOAD

[Pandas] Query 함수 사용법 (Pandas의 꽃)
https://m.blog.naver.com/wideeyed/221867273249

[python/pandas] DataFrame 합치기 : merge, join
출처: https://engineer-mole.tistory.com/208 [매일 꾸준히, 더 깊이:티스토리]



---



In [ ]:
DATA_DIR = "./dataset/uw-madison-gi-tract-image-segmentation/"

In [ ]:
import os
import pandas as pd
from glob import glob

TRAIN_CSV = os.path.join(DATA_DIR, "train.csv")
train_df = pd.read_csv(TRAIN_CSV)

#  glob 모듈의 glob 함수는 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환한다. 단, 조건에 정규식을 사용할 수 없으며
all_train_images = glob(os.path.join(DATA_DIR, "train", "**", "*.png"), recursive=True)

In [ ]:
def get_filepath_from_partial_identifier(_ident, file_list):
    return [x for x in file_list if _ident in x][0]

# csv로 불러온 df를 preprocessing
def df_preprocessing(df, globbed_file_list, is_test=False):
    """ The preprocessing steps applied to get column information """
    # 1. Get Case-ID as a column (str and int)
    # 1. "case_id_str" 컬럼 생성. ex.id = case123_day20_slice_0001 -> case_id_str = case_123
    df["case_id_str"] = df["id"].apply(lambda x: x.split("_", 2)[0])

    # 2. Get Day as a column
    # 2. "day_num_str" 컬럼 생성. ex. day20
    df["day_num_str"] = df["id"].apply(lambda x: x.split("_", 2)[1])

    # 3. Get Slice Identifier as a column
    # 3. "slice_id" 컬럼 생성. ex, slice_0001
    df["slice_id"] = df["id"].apply(lambda x: x.split("_", 2)[2])

    # 4. Get full file paths for the representative scans
    # 4. "f_path" 컬럼 생성
    df["_partial_ident"] = (globbed_file_list[0].rsplit("/", 4)[0]+"/"+ # /kaggle/input/uw-madison-gi-tract-image-segmentation/train/
                           df["case_id_str"]+"/"+ # .../case###/
                           df["case_id_str"]+"_"+df["day_num_str"]+ # .../case###_day##/
                           "/scans/"+df["slice_id"]) # .../slice_#### 
    _tmp_merge_df = pd.DataFrame({"_partial_ident":[x.rsplit("_",4)[0] for x in globbed_file_list], "f_path":globbed_file_list})
    df = df.merge(_tmp_merge_df, on="_partial_ident").drop(columns=["_partial_ident"])

    # 5. Get slice dimensions from filepath (int in pixels)
    # 5. f_path 컬럼으로 부터 slice_w, slice_h 컬럼 생성. 1.5_1.5, 1.63_1.63
    df["slice_w"] = df["f_path"].apply(lambda x: int(x[:-4].rsplit("_",4)[2]))
    df["slice_h"] = df["f_path"].apply(lambda x: int(x[:-4].rsplit("_",4)[1]))

    # 6. Pixel spacing from filepath (float in mm)
    df["px_spacing_h"] = df["f_path"].apply(lambda x: float(x[:-4].rsplit("_",4)[3]))
    df["px_spacing_w"] = df["f_path"].apply(lambda x: float(x[:-4].rsplit("_",4)[4]))

    # is_test가 아니면, False이면, -> 기본값
    if not is_test:
        # 7. Merge 3 Rows Into A Single Row (As This/Segmentation-RLE Is The Only Unique Information Across Those Rows)
        l_bowel_df = df[df["class"]=="large_bowel"][["id", "segmentation"]].rename(columns={"segmentation":"lb_seg_rle"})
        s_bowel_df = df[df["class"]=="small_bowel"][["id", "segmentation"]].rename(columns={"segmentation":"sb_seg_rle"})
        stomach_df = df[df["class"]=="stomach"][["id", "segmentation"]].rename(columns={"segmentation":"st_seg_rle"})
        df = df.merge(l_bowel_df, on="id", how="left")
        df = df.merge(s_bowel_df, on="id", how="left")
        df = df.merge(stomach_df, on="id", how="left")
        df = df.drop_duplicates(subset=["id",]).reset_index(drop=True)

    # 8. Reorder columns to the a new ordering (drops class and segmentation as no longer necessary)
    new_col_order = ["id", "f_path",
                     "lb_seg_rle",
                     "sb_seg_rle", 
                     "st_seg_rle",
                     "slice_h", "slice_w", "px_spacing_h", 
                     "px_spacing_w", "case_id_str", 
                     "day_num_str", "slice_id",]
    if is_test: new_col_order.insert(1, "class")
    new_col_order = [_c for _c in new_col_order if _c in df.columns]
    df = df[new_col_order]
    
    return df

In [ ]:
train_df

,id,class,segmentation
0,case123_day20_slice_0001,large_bowel,NaN
1,case123_day20_slice_0001,small_bowel,NaN
2,case123_day20_slice_0001,stomach,NaN
3,case123_day20_slice_0002,large_bowel,NaN
4,case123_day20_slice_0002,small_bowel,NaN
...,...,...,...
115483,case30_day0_slice_0143,small_bowel,NaN
115484,case30_day0_slice_0143,stomach,NaN
115485,case30_day0_slice_0144,large_bowel,NaN
115486,case30_day0_slice_0144,small_bowel,NaN


# 에러 발생

ValueError: could not convert string to float: '1.50 (1)' 

- value error 발생
- 위치: 함수 df_preprocessing 과정 중 


```
df["px_spacing_w"] = df["f_path"].apply(lambda x: float(x[:-4].rsplit("_",4)[4]))

```


In [ ]:
train_df = df_preprocessing(train_df, all_train_images)
train_df

ValueError: ignored

 ### 에러 원인 찾아보기

In [ ]:
train_df = pd.read_csv(TRAIN_CSV)

문제가 발생한 df_preprocessing 안의 기능들을 한줄씩 실행해봄.

In [ ]:
# 1. Get Case-ID as a column (str and int)
train_df["case_id_str"] = train_df["id"].apply(lambda x: x.split("_", 2)[0])

# 2. Get Day as a column
train_df["day_num_str"] = train_df["id"].apply(lambda x: x.split("_", 2)[1])

# 3. Get Slice Identifier as a column
train_df["slice_id"] = train_df["id"].apply(lambda x: x.split("_", 2)[2])

# 4. Get full file paths for the representative scans
train_df["_partial_ident"] = (all_train_images[0].rsplit("/", 4)[0]+"/"+ # /kaggle/input/uw-madison-gi-tract-image-segmentation/train/
                        train_df["case_id_str"]+"/"+ # .../case###/
                        train_df["case_id_str"]+"_"+train_df["day_num_str"]+ # .../case###_day##/
                        "/scans/"+train_df["slice_id"]) # .../slice_#### 
_tmp_merge_df = pd.DataFrame({"_partial_ident":[x.rsplit("_",4)[0] for x in all_train_images], "f_path":all_train_images})
train_df = train_df.merge(_tmp_merge_df, on="_partial_ident").drop(columns=["_partial_ident"])

In [ ]:
# df의 preprocessing 과정 확인
train_df

,id,class,segmentation,case_id_str,day_num_str,slice_id,f_path
0,case123_day20_slice_0001,large_bowel,NaN,case123,day20,slice_0001,./dataset/uw-madison-gi-tract-image-segmentati...
1,case123_day20_slice_0001,small_bowel,NaN,case123,day20,slice_0001,./dataset/uw-madison-gi-tract-image-segmentati...
2,case123_day20_slice_0001,stomach,NaN,case123,day20,slice_0001,./dataset/uw-madison-gi-tract-image-segmentati...
3,case123_day20_slice_0002,large_bowel,NaN,case123,day20,slice_0002,./dataset/uw-madison-gi-tract-image-segmentati...
4,case123_day20_slice_0002,small_bowel,NaN,case123,day20,slice_0002,./dataset/uw-madison-gi-tract-image-segmentati...
...,...,...,...,...,...,...,...
115666,case30_day0_slice_0143,small_bowel,NaN,case30,day0,slice_0143,./dataset/uw-madison-gi-tract-image-segmentati...
115667,case30_day0_slice_0143,stomach,NaN,case30,day0,slice_0143,./dataset/uw-madison-gi-tract-image-segmentati...
115668,case30_day0_slice_0144,large_bowel,NaN,case30,day0,slice_0144,./dataset/uw-madison-gi-tract-image-segmentati...
115669,case30_day0_slice_0144,small_bowel,NaN,case30,day0,slice_0144,./dataset/uw-madison-gi-tract-image-segmentati...


In [ ]:
# 5. Get slice dimensions from filepath (int in pixels)
train_df["slice_w"] = train_df["f_path"].apply(lambda x: int(x[:-4].rsplit("_",4)[2]))
train_df["slice_h"] = train_df["f_path"].apply(lambda x: int(x[:-4].rsplit("_",4)[1]))

In [ ]:
# preprocessing 과정확인
train_df

,id,class,segmentation,case_id_str,day_num_str,slice_id,f_path,slice_w,slice_h
0,case123_day20_slice_0001,large_bowel,NaN,case123,day20,slice_0001,./dataset/uw-madison-gi-tract-image-segmentati...,266,266
1,case123_day20_slice_0001,small_bowel,NaN,case123,day20,slice_0001,./dataset/uw-madison-gi-tract-image-segmentati...,266,266
2,case123_day20_slice_0001,stomach,NaN,case123,day20,slice_0001,./dataset/uw-madison-gi-tract-image-segmentati...,266,266
3,case123_day20_slice_0002,large_bowel,NaN,case123,day20,slice_0002,./dataset/uw-madison-gi-tract-image-segmentati...,266,266
4,case123_day20_slice_0002,small_bowel,NaN,case123,day20,slice_0002,./dataset/uw-madison-gi-tract-image-segmentati...,266,266
...,...,...,...,...,...,...,...,...,...
115666,case30_day0_slice_0143,small_bowel,NaN,case30,day0,slice_0143,./dataset/uw-madison-gi-tract-image-segmentati...,266,266
115667,case30_day0_slice_0143,stomach,NaN,case30,day0,slice_0143,./dataset/uw-madison-gi-tract-image-segmentati...,266,266
115668,case30_day0_slice_0144,large_bowel,NaN,case30,day0,slice_0144,./dataset/uw-madison-gi-tract-image-segmentati...,266,266
115669,case30_day0_slice_0144,small_bowel,NaN,case30,day0,slice_0144,./dataset/uw-madison-gi-tract-image-segmentati...,266,266


In [ ]:
# 6. Pixel spacing from filepath (float in mm)
train_df["px_spacing_h"] = train_df["f_path"].apply(lambda x: float(x[:-4].rsplit("_",4)[3]))
train_df["px_spacing_w"] = train_df["f_path"].apply(lambda x: float(x[:-4].rsplit("_",4)[4]))
train_df

ValueError: ignored

#### 에러 발견, 지점 확인
- train_df 를 확인해보면 "px_spacing_h" 컬럼까지는 생성이되었다.
- "px_spacing_h"컬럼과 에러가 발생하는 "px_spacing_w" 컬럼은 프로세싱 과정이 크게 다르지 않은데도 문제가 발생한것.
- 더 확실한 확인을 위해서 rsplit이 적용되는 f_path컬럼의 한 데이터를 가져와 확인해보기로 함.


In [ ]:
train_df.head()

,id,class,segmentation,case_id_str,day_num_str,slice_id,f_path,slice_w,slice_h,px_spacing_h
0,case123_day20_slice_0001,large_bowel,NaN,case123,day20,slice_0001,./dataset/uw-madison-gi-tract-image-segmentati...,266,266,1.5
1,case123_day20_slice_0001,small_bowel,NaN,case123,day20,slice_0001,./dataset/uw-madison-gi-tract-image-segmentati...,266,266,1.5
2,case123_day20_slice_0001,stomach,NaN,case123,day20,slice_0001,./dataset/uw-madison-gi-tract-image-segmentati...,266,266,1.5
3,case123_day20_slice_0002,large_bowel,NaN,case123,day20,slice_0002,./dataset/uw-madison-gi-tract-image-segmentati...,266,266,1.5
4,case123_day20_slice_0002,small_bowel,NaN,case123,day20,slice_0002,./dataset/uw-madison-gi-tract-image-segmentati...,266,266,1.5


In [ ]:
x= train_df["f_path"][0]
train_df["f_path"][0]

'./dataset/uw-madison-gi-tract-image-segmentation/train/case123/case123_day20/scans/slice_0001_266_266_1.50_1.50.png'

resplit으로  나눠지는 요소들 확인

In [ ]:
x[:-4].rsplit("_",4)

['./dataset/uw-madison-gi-tract-image-segmentation/train/case123/case123_day20/scans/slice_0001',
 '266',
 '266',
 '1.50',
 '1.50']

#### 과정
- 위의 요소를 확인해보니 기능적으로 문제가 발생한것같진않아서 직접 내용물을 찾아보고자함.
- 오류 메세지의 "1.50 (1)" 이 어느 데이터인지 직접 눈으로 확인해보고자함.
- 초반엔 "(1)" 이 들어간 데이터를 찾지못해서 에러가 된 "rsplit("_",4)[4]" 의 종류들을 확인해봄

In [ ]:
train_df["f_path"].unique()

array(['./dataset/uw-madison-gi-tract-image-segmentation/train/case123/case123_day20/scans/slice_0001_266_266_1.50_1.50.png',
       './dataset/uw-madison-gi-tract-image-segmentation/train/case123/case123_day20/scans/slice_0002_266_266_1.50_1.50.png',
       './dataset/uw-madison-gi-tract-image-segmentation/train/case123/case123_day20/scans/slice_0003_266_266_1.50_1.50.png',
       ...,
       './dataset/uw-madison-gi-tract-image-segmentation/train/case30/case30_day0/scans/slice_0142_266_266_1.50_1.50.png',
       './dataset/uw-madison-gi-tract-image-segmentation/train/case30/case30_day0/scans/slice_0143_266_266_1.50_1.50.png',
       './dataset/uw-madison-gi-tract-image-segmentation/train/case30/case30_day0/scans/slice_0144_266_266_1.50_1.50.png'],
      dtype=object)

In [ ]:
train_df["f_path"].apply(lambda x: x[:-4].rsplit("_",4)[4]).unique()

array(['1.50', '1.63', '1.50 (1)', '1.50 (2)'], dtype=object)

소괄호 검색이 잘 되지않았으니 검색하기 쉬운 1.63부터 3600 행이 있는 것을 확인후, 분제이 특수기호가 들어간 행이있는지 탐색해봄.


특정문자를 포함하는 행추출 
    

https://zephyrus1111.tistory.com/46

In [ ]:
train_df[train_df["f_path"].str.contains('1.50_1.50')==False & (train_df["f_path"].str.contains('1.63_1.63')==False)]

In [ ]:
train_df[train_df["f_path"].str.contains('./dataset/uw-madison-gi-tract-image-segmentation/train/case74/case74_day15/scans/slice_0098_266_266_1.50_1.50')]

,id,class,segmentation,case_id_str,day_num_str,slice_id,f_path,slice_w,slice_h,px_spacing_h
112920,case74_day15_slice_0098,large_bowel,22263 5 22523 13 22545 4 22788 15 22808 13 230...,case74,day15,slice_0098,./dataset/uw-madison-gi-tract-image-segmentati...,266,266,1.5
112921,case74_day15_slice_0098,large_bowel,22263 5 22523 13 22545 4 22788 15 22808 13 230...,case74,day15,slice_0098,./dataset/uw-madison-gi-tract-image-segmentati...,266,266,1.5
112922,case74_day15_slice_0098,large_bowel,22263 5 22523 13 22545 4 22788 15 22808 13 230...,case74,day15,slice_0098,./dataset/uw-madison-gi-tract-image-segmentati...,266,266,1.5
112923,case74_day15_slice_0098,small_bowel,27857 6 28122 8 28386 10 28651 11 28915 13 291...,case74,day15,slice_0098,./dataset/uw-madison-gi-tract-image-segmentati...,266,266,1.5
112924,case74_day15_slice_0098,small_bowel,27857 6 28122 8 28386 10 28651 11 28915 13 291...,case74,day15,slice_0098,./dataset/uw-madison-gi-tract-image-segmentati...,266,266,1.5
112925,case74_day15_slice_0098,small_bowel,27857 6 28122 8 28386 10 28651 11 28915 13 291...,case74,day15,slice_0098,./dataset/uw-madison-gi-tract-image-segmentati...,266,266,1.5
112926,case74_day15_slice_0098,stomach,21967 3 22230 7 22494 10 22759 10 23025 10 232...,case74,day15,slice_0098,./dataset/uw-madison-gi-tract-image-segmentati...,266,266,1.5
112927,case74_day15_slice_0098,stomach,21967 3 22230 7 22494 10 22759 10 23025 10 232...,case74,day15,slice_0098,./dataset/uw-madison-gi-tract-image-segmentati...,266,266,1.5
112928,case74_day15_slice_0098,stomach,21967 3 22230 7 22494 10 22759 10 23025 10 232...,case74,day15,slice_0098,./dataset/uw-madison-gi-tract-image-segmentati...,266,266,1.5


In [ ]:
train_df[(train_df["f_path"].apply(lambda x: x[:-4].rsplit("_",4)[4]) == '1.50 (1)') == True]#.iloc[0].f_path
# (1) 이 180
# (2) 가 3개

,id,class,segmentation,case_id_str,day_num_str,slice_id,f_path,slice_w,slice_h,px_spacing_h
30288,case117_day0_slice_0001,large_bowel,NaN,case117,day0,slice_0001,./dataset/uw-madison-gi-tract-image-segmentati...,266,266,1.5
30290,case117_day0_slice_0001,small_bowel,NaN,case117,day0,slice_0001,./dataset/uw-madison-gi-tract-image-segmentati...,266,266,1.5
30292,case117_day0_slice_0001,stomach,NaN,case117,day0,slice_0001,./dataset/uw-madison-gi-tract-image-segmentati...,266,266,1.5
30339,case117_day0_slice_0017,large_bowel,NaN,case117,day0,slice_0017,./dataset/uw-madison-gi-tract-image-segmentati...,266,266,1.5
30341,case117_day0_slice_0017,small_bowel,NaN,case117,day0,slice_0017,./dataset/uw-madison-gi-tract-image-segmentati...,266,266,1.5
...,...,...,...,...,...,...,...,...,...,...
112991,case74_day15_slice_0117,small_bowel,27044 2 27308 4 27573 5 27838 7 28102 9 28367 ...,case74,day15,slice_0117,./dataset/uw-madison-gi-tract-image-segmentati...,266,266,1.5
112993,case74_day15_slice_0117,stomach,NaN,case74,day15,slice_0117,./dataset/uw-madison-gi-tract-image-segmentati...,266,266,1.5
113016,case74_day15_slice_0125,large_bowel,NaN,case74,day15,slice_0125,./dataset/uw-madison-gi-tract-image-segmentati...,266,266,1.5
113018,case74_day15_slice_0125,small_bowel,NaN,case74,day15,slice_0125,./dataset/uw-madison-gi-tract-image-segmentati...,266,266,1.5


In [ ]:
# case74 day15 slice98 의 (1) 과 (2)는 확인이 됐다. 이 파일이 중복파일인지 알아보기위해 1.50_1.50도 있는지 확인
train_df[train_df["f_path"].str.contains('./dataset/uw-madison-gi-tract-image-segmentation/train/case74/case74_day15/scans/slice_0098_266_266_1.50_1.50')].iloc[2].f_path

'./dataset/uw-madison-gi-tract-image-segmentation/train/case74/case74_day15/scans/slice_0098_266_266_1.50_1.50.png'

In [ ]:
try:
  train_df["px_spacing_w"] = train_df["f_path"].apply(lambda x: float(x[:-4].rsplit("_",4)[4]))
except ValueError as e:
  print(e)
  print('예외가 발생했습니다')

could not convert string to float: '1.50 (1)'
예외가 발생했습니다


- 문제가있는 부분인 fath를 가져온 리스트를 살펴보기로했다.
 - 어떤 과정에 의해서 df에 생성이 잘못된건지 원 리스트부터 문제인지 판별하기위함이었다.
- 문제가 있는 case74_day15/scans/slice_0098 파일들의 인덱스를 탐색해 내용물을 확인했다.

In [ ]:
len(all_train_images)

38557

In [ ]:
# #리스트 컴프리헨션 사용 : matching = [s for s in fruit_list if "베리" in s] 
[i for i in range(len(all_train_images)) if 'case74_day15/scans/slice_0098' in all_train_images[i]]

[26850, 26882, 26895]

In [ ]:
all_train_images[26895]

'./dataset/uw-madison-gi-tract-image-segmentation/train/case74/case74_day15/scans/slice_0098_266_266_1.50_1.50.png'

#### 결론 
- 리스트에서 (1),(2) 파일이 있었다.
- 하나의 파일의 path에서 같은 게 3개 생겨서 (1), (2) 가 붙은것같다.   
(왜지...?   )
 - 에러의 원인은 이것 때문에 float()가 안되어서인데 왜 이런 중복 파일이 생긴겼던 건지는 모르겠다. 
 - 데이터가 저장된 폴더를 살펴봤지만 파일은 한개뿐이었다. 한개의 파일에 3개의 문자열이 생긴것이다.
 - 세그멘테이션등 내용도 확인해봤을때 괄호가 붙은 자료는 확실히 중복자료였다.
- 일단 문제가 되는 문자열을 제거하면 오류가 없어질것이라 예상.
- 중복으로 생성된걸로 보고, 리스트안에서 제거해주기로..

In [ ]:
len([i for i in range(len(all_train_images)) if '(' in all_train_images[i]])

61

61x3 = 183   
( 한파일에 장기가 3개이기때문에)

위에서 df에서 확인한 행의 갯수와 일치한다.   
개수는 제대로 파악한듯한다.

In [ ]:
[s for s in all_train_images if '(' in s]

In [ ]:
len([s for s in all_train_images if '(' not in s])

38496

In [ ]:
len(all_train_images)

38557

In [ ]:
train_df = pd.read_csv(TRAIN_CSV)
all_train_images2 = [s for s in all_train_images if '(' not in s]

#### 해결!

In [ ]:
train_df = df_preprocessing(train_df, all_train_images2)
train_df

,id,f_path,lb_seg_rle,sb_seg_rle,st_seg_rle,slice_h,slice_w,px_spacing_h,px_spacing_w,case_id_str,day_num_str,slice_id
0,case123_day20_slice_0001,./dataset/uw-madison-gi-tract-image-segmentati...,NaN,NaN,NaN,266,266,1.5,1.5,case123,day20,slice_0001
1,case123_day20_slice_0002,./dataset/uw-madison-gi-tract-image-segmentati...,NaN,NaN,NaN,266,266,1.5,1.5,case123,day20,slice_0002
2,case123_day20_slice_0003,./dataset/uw-madison-gi-tract-image-segmentati...,NaN,NaN,NaN,266,266,1.5,1.5,case123,day20,slice_0003
3,case123_day20_slice_0004,./dataset/uw-madison-gi-tract-image-segmentati...,NaN,NaN,NaN,266,266,1.5,1.5,case123,day20,slice_0004
4,case123_day20_slice_0005,./dataset/uw-madison-gi-tract-image-segmentati...,NaN,NaN,NaN,266,266,1.5,1.5,case123,day20,slice_0005
...,...,...,...,...,...,...,...,...,...,...,...,...
38491,case30_day0_slice_0140,./dataset/uw-madison-gi-tract-image-segmentati...,NaN,NaN,NaN,266,266,1.5,1.5,case30,day0,slice_0140
38492,case30_day0_slice_0141,./dataset/uw-madison-gi-tract-image-segmentati...,NaN,NaN,NaN,266,266,1.5,1.5,case30,day0,slice_0141
38493,case30_day0_slice_0142,./dataset/uw-madison-gi-tract-image-segmentati...,NaN,NaN,NaN,266,266,1.5,1.5,case30,day0,slice_0142
38494,case30_day0_slice_0143,./dataset/uw-madison-gi-tract-image-segmentati...,NaN,NaN,NaN,266,266,1.5,1.5,case30,day0,slice_0143
